<a href="https://colab.research.google.com/github/K8rly/GoVote/blob/SarahDivakarla/GoVote_ML_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sb
from pathlib import Path

In [2]:
from google.colab import files
uploaded = files.upload()

Saving voter_2016cleaned.csv to voter_2016cleaned.csv


In [3]:
# 1. Create a DataFrame for the 2016/2020 Election data. 
file_to_open = ("voter_2016cleaned.csv")
ml_2016_df = pd.read_csv(file_to_open,  encoding="ISO-8859-1")
ml_2016_df.head()

,Voter_ID,Birth_Year,City,Zip_Code,Party,Registration_Date,2020_Primary_Election,2020_General_Election,2016_Primary_Election,2016_General_Election,Generational_Group
0,704844,1959,1,44131,3,1901-01-01,3,1,0,1,2
1,1658758,1968,2,44130,1,1996-09-09,3,1,1,1,3
2,2005536,1976,3,44147,1,2004-08-03,3,1,1,1,3
3,2465762,1991,4,44139,3,2010-09-27,3,0,3,0,4
4,710633,1968,5,44131,1,1901-01-01,3,1,1,1,3


In [5]:
# Set voter_id_org column as index
ml_2016_df= ml_2016_df.set_index('Voter_ID')

In [7]:
ml_2016_df.dtypes

Birth_Year                int64
City                      int64
Zip_Code                  int64
Party                     int64
Registration_Date        object
2020_Primary_Election     int64
2020_General_Election     int64
2016_Primary_Election     int64
2016_General_Election     int64
Generational_Group        int64
dtype: object

In [8]:
ml_2016_df['Registration_Date'] =  pd.to_datetime(ml_2016_df['Registration_Date'], dayfirst=True)


In [9]:
# Restrict to voters eligable to vote in 2016 and 2020 election (born 1998 or before)
ml_2016_df['Birth_Year'].describe()

count    555189.000000
mean       1967.552437
std          17.518863
min        1928.000000
25%        1954.000000
50%        1967.000000
75%        1983.000000
max        1998.000000
Name: Birth_Year, dtype: float64

In [11]:
# restrict voters to registration date of 10-5-2016 (registered prior to 2016 election)
ml_2016_df['Registration_Date'].describe()

count                  555189
unique                  11967
top       1901-01-01 00:00:00
freq                   123065
first     1800-01-01 00:00:00
last      2016-10-05 00:00:00
Name: Registration_Date, dtype: object

In [12]:
ml_2016_df = ml_2016_df.drop(['Registration_Date'], axis=1)
ml_2016_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555189 entries, 704844 to 276903
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   Birth_Year             555189 non-null  int64
 1   City                   555189 non-null  int64
 2   Zip_Code               555189 non-null  int64
 3   Party                  555189 non-null  int64
 4   2020_Primary_Election  555189 non-null  int64
 5   2020_General_Election  555189 non-null  int64
 6   2016_Primary_Election  555189 non-null  int64
 7   2016_General_Election  555189 non-null  int64
 8   Generational_Group     555189 non-null  int64
dtypes: int64(9)
memory usage: 42.4 MB


In [15]:
X = ml_2016_df.drop("2020_General_Election",axis=1)
y = ml_2016_df["2020_General_Election"]

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=33, stratify=y)
X_train.shape

(416391, 8)

In [18]:
from sklearn.svm import SVC
model = SVC(kernel='linear')

In [19]:
model.fit(X_train, y_train)

SVC(kernel='linear')

In [20]:
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,1,1
1,1,1
2,1,1
3,1,0
4,1,1


In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.756826467240162

In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.05      0.09     35164
           1       0.76      1.00      0.86    103634

    accuracy                           0.76    138798
   macro avg       0.79      0.52      0.48    138798
weighted avg       0.78      0.76      0.67    138798

